In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from config import DB_NAME, DB_URL, DB_PORT, DB_NAME, USERNAME, PASSWORD

# Import Data

In [ ]:
#####################################################################
### !!! This csv import to be replaced by import from DB instance !!!
### !!! For model design only !!!
#####################################################################

# Read CSV data and verify
sparse_covid_df = pd.read_csv('../data/covid/sparse_county_covid.csv',index_col=0)
dense_covid_df = pd.read_csv('../data/covid/dense_county_covid.csv',index_col=0)
weather_df = pd.read_csv('../data/weather/county_weather.csv',index_col=0)

# Transform Data

In [ ]:
# Add population density and total population values
density = {
    'baltimore': 6866,
    'essex': 6168,
    'cook': 5301,
    'union': 5150,
    'norfolk': 5026,
    'nassau': 4954,
    'harrisonburg':4765,
    'fairfax':2454,
    'camden':2289,
    'harris':2700,
    'franklin':2186,
    'marion':2466,
    'dekalb':2482,
    'duval':1305,
    'wake':1377,
    'bexar':1620
}

population = {
    'baltimore': 621342,
    'essex': 755618,
    'cook': 5231351,
    'union': 543976,
    'norfolk': 245782,
    'nassau': 74629,
    'harrisonburg':50981,
    'fairfax':1118602,
    'camden':513539,
    'harris':4253700,
    'franklin':1195537,
    'marion':918977,
    'dekalb':707089,
    'duval':879602,
    'wake':952151,
    'bexar':1785704
}


In [ ]:
# Add pop_density and population columns to covid data
sparse_covid_df['population'] = pd.Series(dtype=int)
sparse_covid_df['pop_density'] = pd.Series(dtype=int)

dense_covid_df['population'] = pd.Series(dtype=int)
dense_covid_df['pop_density'] = pd.Series(dtype=int)

# Set population and density columns for sparse counties
for i in range(len(sparse_covid_df.index)):
    county = sparse_covid_df.iloc[i,1]

    sparse_covid_df.iloc[i,7] = population[county]
    sparse_covid_df.iloc[i,8] = density[county]

# Set population and density columns for dense counties
for i in range(len(dense_covid_df.index)):
    county = dense_covid_df.iloc[i,1]

    dense_covid_df.iloc[i,7] = population[county]
    dense_covid_df.iloc[i,8] = density[county]

In [ ]:
# Add past_delta14 columns to sparse and dense counties
sparse_covid_df['past_delta14'] = pd.Series(dtype=int)
dense_covid_df['past_delta14'] = pd.Series(dtype=int)

# Sparse county past_delta_14
for i in range(14,len(sparse_covid_df.index)):

    # Set past_delta_14
    past_cases = sparse_covid_df.iloc[i-14,3]
    present_cases = sparse_covid_df.iloc[i,3]
    delta14 = present_cases - past_cases

    sparse_covid_df.iloc[i,9] = delta14

# Dense county past delta14
for i in range(14,len(dense_covid_df.index)):

    # Set past_delta_14
    past_cases = dense_covid_df.iloc[i-14,3]
    present_cases = dense_covid_df.iloc[i,3]
    delta14 = present_cases - past_cases

    dense_covid_df.iloc[i,9] = delta14

In [ ]:
# Sort weather data according to the sparse or dense county sets
sparse_counties = sparse_covid_df['county'].unique()
dense_counties = dense_covid_df['county'].unique()

# Sort weather for each set
sparse_weather_df = weather_df[weather_df['county'].isin(sparse_counties)]
dense_weather_df = weather_df[weather_df['county'].isin(dense_counties)]

In [ ]:
# Merge weather data on sparse and dense covid data
sparse_weather_covid_df = sparse_covid_df.merge(sparse_weather_df,on=['date','county'])
dense_weather_covid_df = dense_covid_df.merge(dense_weather_df,on=['date','county'])

In [ ]:
# Verify Sparse Data
sparse_weather_covid_df.head()

In [ ]:
# Verify Dense Data
dense_weather_covid_df.head()

# Run Models

In [ ]:
# Set Feature categories
weather_features = ['temp_mean(C)','precip_sum(mm)','wind_max(km/h)','min_humidity(%)','max_humidity(%)','mean_humidity(%)']
county_features = ['population','pop_density']
covid_features = ['new_cases','past_delta14']
target = ['future_delta14']

## Sparse Data Model
### All Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use All Features
X = sparse[weather_features + covid_features + county_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case prediction model
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("---------------- All Features -------------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")


### Only Weather Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use Only Weather Features
X = sparse[weather_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case 
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

print("------- Sparse 14-day New Case Prediction -------")
print()
print("-------------- Weather Features -----------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")

### Only Covid Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use Only Covid Features
X = sparse[covid_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case prediction model
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

print("------- Sparse 14-day New Case Prediction -------")
print()
print("---------------- Covid Features -----------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")

### Only County Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use County Features
X = sparse[county_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case prediction model
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

print("------- Sparse 14-day New Case Prediction -------")
print()
print("--------------- County Features -----------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")

### Only County / Weather Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use Only County / Weather Features
X = sparse[weather_features + county_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case prediction model
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

print("------- Sparse 14-day New Case Prediction -------")
print()
print("---------- County / Weather Features ------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")

### Only Covid / Weather Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use Only Covid / Weather Features
X = sparse[weather_features + covid_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case prediction model
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

print("------- Sparse 14-day New Case Prediction -------")
print()
print("---------- Covid / Weather Features -------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")

### Only County / Covid Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
sparse = sparse_weather_covid_df.dropna()

# Use Only County / Covid Features
X = sparse[covid_features + county_features]
y = sparse[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
sparse_model = LinearRegression()

# Sparse 14-day new case 
sparse_model.fit(X_train,y_train)
training_score = sparse_model.score(X_train,y_train)
testing_score = sparse_model.score(X_test,y_test)

print("------- Sparse 14-day New Case Prediction -------")
print()
print("----------- County / Covid Features -------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{sparse_model.feature_names_in_[i]} : {sparse_model.coef_[0][i]}")
print(f"y-intercept : {sparse_model.intercept_[0]}")

## Dense Data Model
### All Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use All features
X = dense[weather_features + covid_features + county_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print("---------------- All Features -------------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")


### Only Weather Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use Only Weather Features
X = dense[weather_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("-------------- Weather Features -----------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")


### Only Covid Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use Only Covid Features
X = dense[covid_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("---------------- Covid Features -----------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")


### Only County Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use Only County Features
X = dense[county_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("--------------- County Features -----------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")


### Only County / Weather Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use Only County / Weather Features
X = dense[weather_features + county_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("---------- County / Weather Features ------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")


### Only Covid / Weather Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use Only Covid / Weather Features
X = dense[weather_features + covid_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("---------- Covid / Weather Features -------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")


### Only County / Covid Features

In [ ]:
# Split data into Training and Testing Sets
# Drop rows with null data
dense = dense_weather_covid_df.dropna()

# Use Only County / Covid Features
X = dense[covid_features + county_features]
y = dense[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

# Use linear regression model
dense_model = LinearRegression()

# Dense 14-day new case prediction model
dense_model.fit(X_train,y_train)
training_score = dense_model.score(X_train,y_train)
testing_score = dense_model.score(X_test,y_test)

# View performance, coefficients and y-intercept
print("------- Dense 14-day New Case Prediction --------")
print()
print("----------- County / Covid Features -------------")
print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")
print()
print('----------------- Coefficients ------------------')
for i in range(0,len(X.columns)):
    print(f"{dense_model.feature_names_in_[i]} : {dense_model.coef_[0][i]}")
print(f"y-intercept : {dense_model.intercept_[0]}")
